In [ ]:
import os
import numpy as np
import nibabel as nib
import tensorflow as tf
import cv2
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate, Dropout
from tensorflow.keras.models import Model

class DataGenerator(Sequence):
    def __init__(self, list_IDs, input_dir, batch_size=2, dim=(128, 128, 128), n_channels=4, n_classes=4, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.input_dir = input_dir
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __load_nifti_file(self, filepath):
        scan = nib.load(filepath)
        return scan.get_fdata()
    
    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_temp):
            folder_name = os.path.join(self.input_dir, f'BraTS20_Training_{ID}')
            for modality_index, modality in enumerate(['flair', 't1', 't1ce', 't2']):
                modality_path = os.path.join(folder_name, f'BraTS20_Training_{ID}_{modality}.nii')
                modality_data = self.__load_nifti_file(modality_path)
                resized_data = cv2.resize(modality_data, (self.dim[0], self.dim[1]))
                X[i, :, :, :, modality_index] = resized_data
            
            mask_path = os.path.join(folder_name, f'BraTS20_Training_{ID}_seg.nii')
            mask_data = self.__load_nifti_file(mask_path)
            mask_resized = tf.image.resize(mask_data, (self.dim[0], self.dim[1]))
            y[i] = tf.one_hot(mask_resized, self.n_classes)
        
        return X, y

# Define Dice coefficient metric
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

# U-Net model definition
def unet_model(input_size=(128, 128, 128, 4), n_classes=4):
    inputs = Input(input_size)
    
    # Encoder
    conv1 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)
    
    conv2 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)
    
    conv3 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)
    
    conv4 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling3D(pool_size=(2, 2, 2))(drop4)
    
    # Bottleneck
    conv5 = Conv3D(1024, (3, 3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv3D(1024, (3, 3, 3), activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder
    up6 = UpSampling3D(size=(2, 2, 2))(drop5)
    up6 = Conv3D(512, (2, 2, 2), activation='relu', padding='same')(up6)
    merge6 = concatenate([drop4, up6], axis=4)
    conv6 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(merge6)
    conv6 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(conv6)
    
    up7 = UpSampling3D(size=(2, 2, 2))(conv6)
    up7 = Conv3D(256, (2, 2, 2), activation='relu', padding='same')(up7)
    merge7 = concatenate([conv3, up7], axis=4)
    conv7 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(merge7)
    conv7 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(conv7)
    
    up8 = UpSampling3D(size=(2, 2, 2))(conv7)
    up8 = Conv3D(128, (2, 2, 2), activation='relu', padding='same')(up8)
    merge8 = concatenate([conv2, up8], axis=4)
    conv8 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(merge8)
    conv8 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv8)
    
    up9 = UpSampling3D(size=(2, 2, 2))(conv8)
    up9 = Conv3D(64, (2, 2, 2), activation='relu', padding='same')(up9)
    merge9 = concatenate([conv1, up9], axis=4)
    conv9 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(merge9)
    conv9 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv9)
    conv9 = Conv3D(2, (3, 3, 3), activation='relu', padding='same')(conv9)
    conv10 = Conv3D(n_classes, (1, 1, 1), activation='softmax')(conv9)
    
    model = Model(inputs=inputs, outputs=conv10)
    
    return model

# Prepare IDs and data generators
input_dir = './Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData'
train_and_test_ids = [f.name.split('_')[-1] for f in os.scandir(input_dir) if f.is_dir()]
train_and_test_ids, val_ids = train_test_split(train_and_test_ids, test_size=0.2)
train_ids, test_ids = train_test_split(train_and_test_ids, test_size=0.15, random_state=42)

train_generator = DataGenerator(train_ids, input_dir)
val_generator = DataGenerator(val_ids, input_dir)

# Instantiate and compile the model
model = unet_model()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[dice_coefficient])

# Train the model
model.fit(train_generator, validation_data=val_generator, epochs=50)


In [ ]:
# tqdm
import os
import numpy as np
import nibabel as nib
import tensorflow as tf
import cv2
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate, Dropout
from tensorflow.keras.models import Model
from tqdm import tqdm

class DataGenerator(Sequence):
    def __init__(self, list_IDs, input_dir, batch_size=2, dim=(128, 128, 128), n_channels=4, n_classes=4, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.input_dir = input_dir
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __load_nifti_file(self, filepath):
        scan = nib.load(filepath)
        return scan.get_fdata()
    
    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_temp):
            folder_name = os.path.join(self.input_dir, f'BraTS20_Training_{ID}')
            for modality_index, modality in enumerate(['flair', 't1', 't1ce', 't2']):
                modality_path = os.path.join(folder_name, f'BraTS20_Training_{ID}_{modality}.nii')
                modality_data = self.__load_nifti_file(modality_path)
                resized_data = cv2.resize(modality_data, (self.dim[0], self.dim[1]))
                X[i, :, :, :, modality_index] = resized_data
            
            mask_path = os.path.join(folder_name, f'BraTS20_Training_{ID}_seg.nii')
            mask_data = self.__load_nifti_file(mask_path)
            mask_resized = tf.image.resize(mask_data, (self.dim[0], self.dim[1]))
            y[i] = tf.one_hot(mask_resized, self.n_classes)
        
        return X, y

# Define Dice coefficient metric
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

# U-Net model definition
def unet_model(input_size=(128, 128, 128, 4), n_classes=4):
    inputs = Input(input_size)
    
    # Encoder
    conv1 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)
    
    conv2 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)
    
    conv3 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)
    
    conv4 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling3D(pool_size=(2, 2, 2))(drop4)
    
    # Bottleneck
    conv5 = Conv3D(1024, (3, 3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv3D(1024, (3, 3, 3), activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder
    up6 = UpSampling3D(size=(2, 2, 2))(drop5)
    up6 = Conv3D(512, (2, 2, 2), activation='relu', padding='same')(up6)
    merge6 = concatenate([drop4, up6], axis=4)
    conv6 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(merge6)
    conv6 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(conv6)
    
    up7 = UpSampling3D(size=(2, 2, 2))(conv6)
    up7 = Conv3D(256, (2, 2, 2), activation='relu', padding='same')(up7)
    merge7 = concatenate([conv3, up7], axis=4)
    conv7 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(merge7)
    conv7 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(conv7)
    
    up8 = UpSampling3D(size=(2, 2, 2))(conv7)
    up8 = Conv3D(128, (2, 2, 2), activation='relu', padding='same')(up8)
    merge8 = concatenate([conv2, up8], axis=4)
    conv8 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(merge8)
    conv8 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv8)
    
    up9 = UpSampling3D(size=(2, 2, 2))(conv8)
    up9 = Conv3D(64, (2, 2, 2), activation='relu', padding='same')(up9)
    merge9 = concatenate([conv1, up9], axis=4)
    conv9 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(merge9)
    conv9 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv9)
    conv9 = Conv3D(2, (3, 3, 3), activation='relu', padding='same')(conv9)
    conv10 = Conv3D(n_classes, (1, 1, 1), activation='softmax')(conv9)
    
    model = Model(inputs=inputs, outputs=conv10)
    
    return model

# Prepare IDs and data generators
input_dir = './Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData'
train_and_test_ids = [f.name.split('_')[-1] for f in os.scandir(input_dir) if f.is_dir()]
train_and_test_ids, val_ids = train_test_split(train_and_test_ids, test_size=0.2)
train_ids, test_ids = train_test_split(train_and_test_ids, test_size=0.15, random_state=42)

train_generator = DataGenerator(train_ids, input_dir)
val_generator = DataGenerator(val_ids, input_dir)

# Instantiate and compile the model
model = unet_model()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[dice_coefficient])

# Custom training loop with progress bar
epochs = 50
steps_per_epoch = len(train_generator)
validation_steps = len(val_generator)

for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    with tqdm(total=steps_per_epoch, desc='Training', unit='batch') as pbar:
        for step in range(steps_per_epoch):
            X_batch, y_batch = train_generator[step]
            loss = model.train_on_batch(X_batch, y_batch)
            pbar.set_postfix({'loss': loss[0], 'dice_coefficient': loss[1]})
            pbar.update(1)

    with tqdm(total=validation_steps, desc='Validation', unit='batch') as pbar:
        val_loss = []
        for step in range(validation_steps):
            X_val_batch, y_val_batch = val_generator[step]
            loss = model.test_on_batch(X_val_batch, y_val_batch)
            val_loss.append(loss)
            pbar.set_postfix({'val_loss': loss[0], 'val_dice_coefficient': loss[1]})
            pbar.update(1)

    avg_val_loss = np.mean(val_loss, axis=0)
    print(f'Validation loss: {avg_val_loss[0]}, Validation Dice Coefficient: {avg_val_loss[1]}')



In [ ]:
import os
import numpy as np
import nibabel as nib
import tensorflow as tf
import cv2
from tensorflow.keras.utils import Sequence
from tqdm import tqdm

class NiftiDataGenerator(Sequence):
    def __init__(self, input_dir, total_size, modalities, batch_size=32, dim=(128, 128), n_channels=4, n_classes=4, shuffle=True):
        self.input_dir = input_dir
        self.total_size = total_size
        self.modalities = modalities
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.total_size['Training'] / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_ids = [k for k in indexes]
        X, y = self.__data_generation(batch_ids)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(self.total_size['Training'])
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_ids):
        X = np.empty((self.batch_size * 155, *self.dim, self.n_channels))
        y = np.empty((self.batch_size * 155, *self.dim, self.n_classes), dtype=int)

        with tqdm(total=self.batch_size * 155, desc='Generating batch data', unit='slice') as pbar:
            for i, ID in enumerate(batch_ids):
                id_str = f"{ID + 1:03d}"
                for h in range(155):
                    for k, modality in enumerate(self.modalities):
                        channel_name = f'BraTS20_Training_{id_str}_{modality}'
                        channel_path = os.path.join(self.input_dir, 'Training', modality)
                        nmpy_channel = self.load_nifti_file(os.path.join(channel_path, channel_name + '.nii'))
                        X[i * 155 + h, :, :, k] = cv2.resize(nmpy_channel[:, :, h], self.dim)
                    
                    mask_name = f'BraTS20_Training_{id_str}_seg'
                    mask_path = os.path.join(self.input_dir, 'Training', 'mask')
                    npy_mask = self.load_nifti_file(os.path.join(mask_path, mask_name + '.nii'))
                    mask = tf.one_hot(npy_mask[:, :, h], self.n_classes)
                    y[i * 155 + h] = tf.image.resize(mask, self.dim).numpy()

                    pbar.update(1)

        y[y == self.n_classes] = self.n_classes - 1
        return X, y

    def load_nifti_file(self, filepath):
        scan = nib.load(filepath)
        return scan.get_fdata()

input_dir = './Dataset'
total_size = {'Training': 369, 'Validation': 125}
modalities = ['flair', 't1', 't1ce', 't2']
batch_size = 2

training_generator = NiftiDataGenerator(input_dir=input_dir, total_size=total_size, modalities=modalities, batch_size=batch_size)

# Define a model suitable for segmentation
inputs = tf.keras.layers.Input(shape=(128, 128, 4))
c1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
c1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
c2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
c4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)

c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p4)
c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c5)

u6 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u6)
c6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u7)
c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u8)
c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1])
c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u9)
c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c9)

outputs = tf.keras.layers.Conv2D(4, (1, 1), activation='softmax')(c9)

model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(training_generator, epochs=10)
